In [1]:
import sklearn
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.regularizers import l2

In [ ]:
COMMENT = 'comment_text'
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
base_path = 'jigsaw-toxic-comment-classification-challenge'
train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')
submission = pd.read_csv(f'{base_path}/sample_submission.csv')
test_labels = pd.read_csv(f'{base_path}/test_labels.csv')

In [ ]:
EMBEDDING_FILE = f'{base_path}/glove.6B.300d.txt'
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

with open(EMBEDDING_FILE) as f:
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in f)

all_embs = np.stack(embeddings_index.values())
emb_mean = all_embs.mean()
emb_std = all_embs.std()
emb_mean,emb_std

# Problem understanding

# Dataset generation

# Metrics define

# Validation strategy

# Data processing (extract useful information)

In [5]:
MAX_TOKENS = 30000
MAX_SENTENSE_LEN = 100
EMBEDDING_SIZE = all_embs.shape[1]

In [6]:
x_train = train["comment_text"].values
y_train = train[LABELS].values
x_test = test["comment_text"].values

tokenizer = text.Tokenizer(num_words=MAX_TOKENS)
tokenizer.fit_on_texts(list(x_train) + list(x_test)) # is it a proper trick?

x_train, x_test = map(tokenizer.texts_to_sequences, [x_train, x_test])
x_train, x_test = map(lambda x: sequence.pad_sequences(x, maxlen=MAX_SENTENSE_LEN), [x_train, x_test])


In [7]:
word_index = tokenizer.word_index
nb_words = min(MAX_TOKENS, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBEDDING_SIZE))
for word, i in word_index.items():
    if i >= MAX_TOKENS: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

# Data understanding & visualization

# De-noise (no drop data)

# Feature engineering

# Offline augmentation

# Standarization

# Scaling

# Normalization

# Feature selection

# Data selection

# Optimization

In [8]:
MAX_SENTENSE_LEN, MAX_TOKENS, EMBEDDING_SIZE, embedding_matrix

(100,
 30000,
 300,
 array([[-0.44480804, -0.28227118,  0.39783603, ..., -0.2178669 ,
          0.30672413,  0.00205656],
        [ 0.04656   ,  0.21318001, -0.0074364 , ...,  0.0090611 ,
         -0.20988999,  0.053913  ],
        [-0.25756001, -0.057132  , -0.67189997, ..., -0.16043   ,
          0.046744  , -0.070621  ],
        ...,
        [-0.21454   , -0.52314001, -0.1452    , ...,  0.63898998,
          0.38471001,  0.47398999],
        [-0.18870001, -0.56199002, -0.026557  , ...,  0.16516   ,
          0.091691  ,  0.55971998],
        [ 0.25328001, -0.83880001, -0.59951001, ..., -0.39467999,
          0.69099998,  0.82291001]]))

In [8]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, weights=[embedding_matrix], embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.GRU(64, return_sequences=True, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.GRU(32, return_sequences=False, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('81.submission.csv', index=False)

# ~ 0.96425

Epoch 1/100


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


593/593 [==============================] - 242s 408ms/step - loss: 9.3609 - acc: 0.7809 - val_loss: 6.4431 - val_acc: 0.9952
Epoch 2/100
593/593 [==============================] - 249s 420ms/step - loss: 4.6999 - acc: 0.9720 - val_loss: 3.3038 - val_acc: 0.9952
Epoch 3/100
593/593 [==============================] - 255s 430ms/step - loss: 2.4234 - acc: 0.9832 - val_loss: 1.7163 - val_acc: 0.9952
Epoch 4/100
593/593 [==============================] - 243s 411ms/step - loss: 1.2655 - acc: 0.9890 - val_loss: 0.9064 - val_acc: 0.9952
Epoch 5/100
593/593 [==============================] - 255s 430ms/step - loss: 0.6760 - acc: 0.9908 - val_loss: 0.4958 - val_acc: 0.9952
Epoch 6/100
593/593 [==============================] - 256s 431ms/step - loss: 0.3772 - acc: 0.9914 - val_loss: 0.2899 - val_acc: 0.9952
Epoch 7/100
593/593 [==============================] - 255s 431ms/step - loss: 0.2264 - acc: 0.9929 - val_loss: 0.1861 - val_acc: 0.9952
Epoch 8/100
593/593 [==============================] 

In [9]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix], mask_zero=True, embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('82.submission.csv', index=False)

# ~ 0.97069

Epoch 1/100


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


593/593 [==============================] - 464s 783ms/step - loss: 9.3014 - acc: 0.8833 - val_loss: 6.4321 - val_acc: 0.9939
Epoch 2/100
593/593 [==============================] - 458s 773ms/step - loss: 4.6854 - acc: 0.9879 - val_loss: 3.2895 - val_acc: 0.9940
Epoch 3/100
593/593 [==============================] - 458s 773ms/step - loss: 2.4076 - acc: 0.9916 - val_loss: 1.7009 - val_acc: 0.9945
Epoch 4/100
593/593 [==============================] - 459s 775ms/step - loss: 1.2512 - acc: 0.9920 - val_loss: 0.8941 - val_acc: 0.9942
Epoch 5/100
593/593 [==============================] - 457s 771ms/step - loss: 0.6635 - acc: 0.9915 - val_loss: 0.4857 - val_acc: 0.9942
Epoch 6/100
593/593 [==============================] - 447s 754ms/step - loss: 0.3663 - acc: 0.9922 - val_loss: 0.2800 - val_acc: 0.9945
Epoch 7/100
593/593 [==============================] - 393s 663ms/step - loss: 0.2165 - acc: 0.9924 - val_loss: 0.1770 - val_acc: 0.9941
Epoch 8/100
593/593 [==============================] 

In [10]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('83.submission.csv', index=False)

# ~ 0.97312

Epoch 1/100
593/593 [==============================] - 391s 660ms/step - loss: 0.0955 - acc: 0.9203 - val_loss: 0.0528 - val_acc: 0.9761
Epoch 2/100
593/593 [==============================] - 382s 644ms/step - loss: 0.0496 - acc: 0.9836 - val_loss: 0.0487 - val_acc: 0.9929
Epoch 3/100
593/593 [==============================] - 380s 640ms/step - loss: 0.0446 - acc: 0.9880 - val_loss: 0.0490 - val_acc: 0.9942
Epoch 00003: early stopping


In [11]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=32,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('84.submission.csv', index=False)

# ~ 0.97945

Epoch 1/100
4738/4738 [==============================] - 1650s 348ms/step - loss: 0.0591 - acc: 0.9534 - val_loss: 0.0470 - val_acc: 0.9934
Epoch 2/100
4738/4738 [==============================] - 1580s 333ms/step - loss: 0.0426 - acc: 0.9864 - val_loss: 0.0443 - val_acc: 0.9926
Epoch 3/100
4738/4738 [==============================] - 1529s 323ms/step - loss: 0.0378 - acc: 0.9790 - val_loss: 0.0455 - val_acc: 0.9902
Epoch 00003: early stopping


In [12]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('85.submission.csv', index=False)

# ~ 0.97899

Epoch 1/100
593/593 [==============================] - 360s 606ms/step - loss: 0.0779 - accuracy: 0.8124 - val_loss: 0.0485 - val_accuracy: 0.9952
Epoch 2/100
593/593 [==============================] - 351s 591ms/step - loss: 0.0446 - accuracy: 0.9594 - val_loss: 0.0468 - val_accuracy: 0.9946
Epoch 3/100
593/593 [==============================] - 300s 506ms/step - loss: 0.0387 - accuracy: 0.9616 - val_loss: 0.0468 - val_accuracy: 0.9921
Epoch 00003: early stopping


In [8]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('86.submission.csv', index=False)
# ~ 0.97773

Epoch 1/100
593/593 [==============================] - 836s 1s/step - loss: 0.0747 - accuracy: 0.8119 - val_loss: 0.0496 - val_accuracy: 0.9950
Epoch 2/100
593/593 [==============================] - 792s 1s/step - loss: 0.0449 - accuracy: 0.9212 - val_loss: 0.0457 - val_accuracy: 0.9935
Epoch 3/100
593/593 [==============================] - 785s 1s/step - loss: 0.0392 - accuracy: 0.9257 - val_loss: 0.0454 - val_accuracy: 0.9920
Epoch 4/100
593/593 [==============================] - 744s 1s/step - loss: 0.0345 - accuracy: 0.9067 - val_loss: 0.0470 - val_accuracy: 0.9863
Epoch 00004: early stopping


# Parameter tuning

# Online augmentation

# Model selection / blending

# Post-processing

# Evaluation

# Reasoning

# Monitoring

In [9]:
ls

101.submission.csv        153.submission.csv
102.submission.csv        154.submission.csv
103.submission.csv        155.submission.csv
104.submission.csv        15.ensemble.ipynb
105.submission.csv        1.EDA_BASELINES.ipynb
106.submission.csv        2.RNN_BASELINES.ipynb
10.DoubleEmbedding.ipynb  3.EDA.ipynb
111.submission.csv        4.TEXT_PROCESS.ipynb
112.submission.csv        5.SentimentFeature_EDA.ipynb
113.submission.csv        6.replace_space.ipynb
114.submission.csv        71.submission.csv
115.submission.csv        72.submission.csv
116.submission.csv        73.submission.csv
11.Preprocess.ipynb       74.submission.csv
121.submission.csv        75.submission.csv
122.submission.csv        76.submission.csv
123.submission.csv        7.glove.ipynb
124.submission.csv        81.submission.csv
125.submission.csv        82.submission.csv
126.submission.csv        83.submission.csv
12.LowerEmbedding.ipynb   84.submission.csv
131.submission.csv        85.submission.csv
132.submissio